# SCHEMA EVOLUTION WITH MERGE SCHEMA COMMAND

In [0]:
from delta.tables import *

In [0]:
DeltaTable.createOrReplace(spark) \
    .tableName("employee") \
    .addColumn("id", "INT") \
    .addColumn("name", "STRING") \
    .addColumn("gender", "STRING") \
    .addColumn("salary", "INT") \
    .addColumn("dept", "STRING") \
    .location('dbfs:/FileStore/tables/delta/merge_schema') \
    .execute()

Out[2]: <delta.tables.DeltaTable at 0x7fec2c5c0220>

In [0]:
%sql

INSERT INTO employee VALUES (1, "Carl Mike", "m", 170000, "Support");
INSERT INTO employee VALUES (2, "Mikel Clark", "m", 254300, "IT");
INSERT INTO employee VALUES (3, "Bob Smith", "m", 220000, "IT");

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

SELECT * FROM employee;

id,name,gender,salary,dept
1,Carl Mike,m,170000,Support
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT


## Schema Evolution

In [0]:
data_employee = [(4, "Mary Scala", "f", 230000, "Sales", True),
                 (5, "Susan Liam", "f", 150000, "Sales", False),
                 (6, "Xi Wuan", "f", 150000, "IT", True)]

schema_employee_v2 = "id INTEGER, name STRING, gender STRING, salary INTEGER, dept STRING, active BOOLEAN"

In [0]:
df_employee_v2 = spark.createDataFrame(data=data_employee,schema=schema_employee_v2)

df_employee_v2.show()

+---+----------+------+------+-----+------+
| id|      name|gender|salary| dept|active|
+---+----------+------+------+-----+------+
|  4|Mary Scala|     f|230000|Sales|  true|
|  5|Susan Liam|     f|150000|Sales| false|
|  6|   Xi Wuan|     f|150000|   IT|  true|
+---+----------+------+------+-----+------+



##Normal Write Fail

In [0]:
df_employee_v2.write.format("delta").mode("append").saveAsTable("employee")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4346387320893596>:1
----> 1 df_employee_v2.write.format("delta").mode("append").saveAsTable("employee")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1518 if format is not None:
   1519     self.format(format)
-> 1520 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self

## Merge Schema

In [0]:
df_employee_v2.write.option("mergeSchema", "true").format("delta").mode("append").saveAsTable("employee")

## Query Combined Schema

In [0]:
%sql

SELECT * FROM employee;

id,name,gender,salary,dept,active
4,Mary Scala,f,230000,Sales,true
5,Susan Liam,f,150000,Sales,false
6,Xi Wuan,f,150000,IT,true
1,Carl Mike,m,170000,Support,null
2,Mikel Clark,m,254300,IT,null
3,Bob Smith,m,220000,IT,null


## New schema update

In [0]:
data_employee_v2 = [(4, "Mary Scala", "f", 230000, 60, True),
                 (5, "Susan Liam", "f", 150000, 40, False),
                 (6, "Xi Wuan", "f", 150000, 23, True)]

schema_employee_v3 = "id INTEGER, name STRING, gender STRING, salary INTEGER, age INTEGER, active BOOLEAN"

In [0]:
df_employee_v3 = spark.createDataFrame(data=data_employee_v2,schema=schema_employee_v3)

df_employee_v3.show()

+---+----------+------+------+---+------+
| id|      name|gender|salary|age|active|
+---+----------+------+------+---+------+
|  4|Mary Scala|     f|230000| 60|  true|
|  5|Susan Liam|     f|150000| 40| false|
|  6|   Xi Wuan|     f|150000| 23|  true|
+---+----------+------+------+---+------+



In [0]:
df_employee_v3.write.option("mergeSchema", "true").format("delta").mode("append").saveAsTable("employee")

In [0]:
%sql

SELECT * FROM employee;

id,name,gender,salary,dept,active,age
4,Mary Scala,f,230000,Sales,true,null
5,Susan Liam,f,150000,Sales,false,null
4,Mary Scala,f,230000,null,true,60
5,Susan Liam,f,150000,null,false,40
6,Xi Wuan,f,150000,null,true,23
6,Xi Wuan,f,150000,IT,true,null
1,Carl Mike,m,170000,Support,null,null
2,Mikel Clark,m,254300,IT,null,null
3,Bob Smith,m,220000,IT,null,null
